# Introduction

The number and severity of cyber-attacks has been increasing at an alarming rate. Currently software security reacts to attacks, trying to minimize damage after it has already occurred. The inherit delay in a reactive approach will inevitably result in larger-than-necessary losses. To prevent or minimize such losses, a proactive approach to security is needed. This proactive approach is the basis of PERCEIVE. By being able to identify potential attacks before they can cause harm, we will be able to finally turn the tide and stop being on the defensive.

Many weaknesses in software are known, and are publicly indexed, but this information is not reaching decision makers. MITRE’s databases—CVE for vulnerabilities, CWE for weaknesses, and CAPEC for attack patterns—hold these past concepts.

While the data in these databases was rich, it was loosely structured and high in volume. My portion of this project was to understand the structure and contents of the CWE database so that we could create a more easily understood and analyzable corpus. This corpus, through machine learning, will be used to compare known concepts (CWEs) to emerging concepts in hacker discussion groups.  To aid in determining similarity between our indexed concepts and emerging concepts, we need to understand the specificity, subject, time and purpose of each concept that we index. By analyzing emerging concepts in this manner, we can direct the attention of software developers, managers, and decision-makers so that they can proactively fix the security weaknesses in an economic and time efficient way.

In order to create this corpus, we must understand the ways in which the aformentioned indexes are organized. This purpose of this notebook is to record and document the way that the CWE database functions. The version of CWE used for the data analytics in this notebook in 2.9. 

# Structure

   The data in CWE is organized in two seperate ways. The first part of this section will deal with the structure and formating of the XML section. XML is language designed to carry information and XML uses fields to hold the information. The objective of this notebook was to determine which fields would be useful for the corpus. The raw XML data can be downloaded at https://cwe.mitre.org/data/. The version used for this notebook is 2.9. 

In [37]:
import lxml.etree
tree = lxml.etree.parse('cwec_v2.9.xml')
root = tree.getroot()
for table in root: 
    print (table.tag)

Views
Categories
Weaknesses
Compound_Elements


There are four main tables in the XML. They are <b>Views</b>, <b>Categories</b>, <b>Weaknesses</b>, and <b>Coumpound_Elements</b>. In the XML the contents of these main categories is messy, however, on the website these main tables follow a strict hierarchy. 

Below is a graphical representation of the XML structure.

<img src="images/CWE_Map.png">

Visualizing the XML data is a headache, however the data for the website is significantly more structured. The four main tables from the XML return, however this time they are organized in a strict hierarchy. There are two <b>Views</b> and they are always at the top of the hierarchy. Every <b>Category</b>, <b>Weakness</b>, and <b>Compound_Element</b> is a "MemberOf" one or both of these two views. <b>Weaknesses</b> are further broken down into three levels. <b>Weakness Class</b> is the top level. <b>Weakness Classes</b> contain descriptions of weaknesses that are very abstract and is the most general of the three <b>Weaknesses</b>. <b>Weakness Bases</b> are the middle ground and contain details on detection and prevention. <b>Weakness Variants</b> are the most specific and are typically limited to a specific language or technology. <b>Categories</b> simply contain entries that share common characteristics. <b> Compound Elements</b> can either be composites or chains as of CWE 2.9. However it is stated that this can change as necessary. 
    

The hierarcy functions as follows. The two <b>Views</b> are the apex of the hierarchy. Every <b>Weakness</b> or <b>Category</b> directly contained within one or both <b>Views</b> is refered as a "MemberOf". This is unique. All <b>Weaknesses</b> <b>Classes</b> and <b>Compound Elements</b> are contained within these views, however not all of them are directly contained within them. For those contained in other <b>Weaknesses</b> they are considered to be the "ChildOf" while their holder is considered the "ParentOf". Any member of the hierachy can be a "ParentOf" any member that is below it or of the same rank. Conversely any member of the hierarcy can be a "ChildOf" any member that is above it or the same rank. The exception to this rule is that a <b>Category</b> can be the "ParentOf" a <b>Weakness Class</b> yet is the "ChildOf" a <b>Weakness Base</b> at least once.   <a href="https://cwe.mitre.org/data/definitions/60.html"> See CWE ID 60 </a>.

<img src="images/CWE_Fields.png">

# Counting Fields

Each weakness contains fields. These fields in turn contain information regarding that weakness. The type of information contained within these fields depends on the type of field. Each type of field has a predetermined format and a specificed topic and type of content. We need to determine which fields will be most useful for the program and direct the programs attention to those fields. The fields that will be useful not only have to contain useful information, but also occur frequently enough to be used regularly. The goal of the scripts below is to determine which fields occur frequently enough to be used. 

The scripts below record all of the fields in the weaknesses. 

In [38]:
weakness_table = root[2]
for row in weakness_table[0]: 
    print (row.tag)

Description
Relationships
Weakness_Ordinalities
Applicable_Platforms
Time_of_Introduction
Common_Consequences
Potential_Mitigations
Causal_Nature
Demonstrative_Examples
Taxonomy_Mappings
Content_History


In [39]:
for row in weakness_table[20]: 
    print (row.tag)

Description
Relationships
Relationship_Notes
Weakness_Ordinalities
Applicable_Platforms
Alternate_Terms
Terminology_Notes
Time_of_Introduction
Likelihood_of_Exploit
Common_Consequences
Detection_Methods
Potential_Mitigations
Causal_Nature
Demonstrative_Examples
Observed_Examples
Functional_Areas
Affected_Resources
References
Taxonomy_Mappings
White_Box_Definitions
Related_Attack_Patterns
Content_History


Now that we have a list of all of the fields, we can count to see how many times they are used. 

In [40]:
histogram = {}
for row in weakness_table: 
    for column in row: 
        if column.tag not in histogram: 
            histogram[column.tag] = 0
        else:
            histogram[column.tag] += 1
print (histogram)

{'Modes_of_Introduction': 32, 'Taxonomy_Mappings': 597, 'White_Box_Definitions': 29, 'Relationships': 705, 'Content_History': 718, 'References': 281, 'Functional_Areas': 27, 'Detection_Methods': 76, 'Enabling_Factors_for_Exploitation': 22, 'Applicable_Platforms': 556, 'Affected_Resources': 50, 'Common_Consequences': 701, 'Other_Notes': 23, 'Relationship_Notes': 122, 'Potential_Mitigations': 523, 'Likelihood_of_Exploit': 184, 'Description': 718, 'Research_Gaps': 74, 'Background_Details': 41, 'Time_of_Introduction': 664, 'Relevant_Properties': 15, 'Weakness_Ordinalities': 130, 'Related_Attack_Patterns': 206, 'Observed_Examples': 357, 'Causal_Nature': 74, 'Demonstrative_Examples': 385, 'Maintenance_Notes': 86, 'Theoretical_Notes': 26, 'Terminology_Notes': 26, 'Alternate_Terms': 65}


Finally in order to make it easier to read, we create a histogram. 

In [41]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook() #So plots

data = {}
data['Entries'] = histogram

df_data = pd.DataFrame(data).sort_values(by='Entries', ascending=True)
series = df_data.loc[:,'Entries']

p = figure(width=800, y_range=series.index.tolist(), title="Weaknesses Histogram")

p.xaxis.axis_label = 'Frequency'
p.xaxis.axis_label_text_font_size = '10pt'
p.xaxis.major_label_text_font_size = '8pt'

p.yaxis.axis_label = 'Field'
p.yaxis.axis_label_text_font_size = '10pt'
p.yaxis.major_label_text_font_size = '8pt'

j = 1
for k,v in series.iteritems():
  
  #Print fields, values, orders
  #print (k,v,j) 
  p.rect(x=v/2, y=j, width=abs(v), height=0.4,
    width_units="data", height_units="data")
  j += 1

Loading BokehJS ...

In [42]:
show(p)

# Analyzing the Fields

Now that we have the frequencies of the fields we can determine which fields will be most useful in creating our corpus. For this purpose the arbitrary number of 100 was chosen. If a field occurs more than 100 times it will be detailed below with an example. The observations are based on limited research and analysis so they are not perfect, but they reasonably describe the rules and regulations of each field.

### Currently Useful Fields

<table>
<th>Name</th><th>Description</th><th  width="50%">Example</th>
<tr>
<td>Description</td><td>A brief description of the Weakness. Typically one to two sentances. Can also contain an Extended Description which goes into further detail. It should be noted that sometimes the extended description is shorter than the regular description but this is rarely the case</td><td><img src="images/Description.png"></td>
</tr>
<tr>
<td>Common_Consequences</td><td>A single sentence. There is also the scope which is a one or two word term as will as the technical impact which varies in format.</td><td><img src="images/Common_Consequences.png"><br><img src="images/Common_Consequences2.PNG"></td> 
</tr>
<tr>
<td>Potential_Mitigations</td><td> Describes how to prevent exploit at various steps in the development cycle. Typically a single sentence per phase and there can be a variable amount of phases</td><td><img src="images/Potential_Mitigations.png"></td> 
</tr>
<tr>
<td>Demonstrative_Examples</td><td>Contains code pertaining to the weakness, but more importantly it can contain extremely detailed descriptions. The length is highly variable but the term density is high. The code is contained within an HTML tag however the notes by whoever wrote the article can be strucutred in a multitude of ways.</td><td><img src="images/Demonstrative_Example.png"><br><img src="images/Demonstrative_Example2.png"></td> 
</tr>
</table>   

### Fields That Might Be Useful Later

<table>
<th>Name</th><th>Description</th><th width="50%">Example</th>
<tr>
<td>Relationships</td><td>Lists other fields fields and relationship to said fields.</td><td><img src="images/Relationships.png"></td>
</tr>
<tr>
<td>Time_of_Introduction</td><td>Gives the stage(s) in the development cycle that this weakness can effect.</td><td><img src="images/Time_Of_Introduction.png"></td>
</tr>
<tr>
<td>Related_Attack_Patterns</td><td>Lists CAPEC entries that relate to topic.</td><td><img src="images/Related_Attack_Patterns.png"></td>
</tr>
<tr>
<td>Observed_Examples</td><td>References CVE entry. See CWE ID number 141</td><td><img src="images/Related_Attack_Patterns.png"></td>
</tr>
<tr>
<td>Taxonomy_Mappings</td><td>Alternative way to organize and understand the data.</td><td><img src="images/Taxonomy_Mappings.png"></td>
</tr>
</table>   

### Currently Discarded Fields

<table>
<th>Name</th><th>Description</th><th width="50%">Example</th>
<tr>
<td>Content_History</td><td>Shows original sumission date along with dates of modifications as well as other relevent data.</td><td><img src="images/Content_History.png"></td>
</tr>
<tr>
<td>Content_History</td><td>Shows original sumission date along with dates of modifications as well as other relevent data.</td><td><img src="images/Content_History.png"></td>
</tr>
<tr>
<td>Application_Platforms</td><td>Gives the languages that the weakness effects.</td><td><img src="images/Application_Platform.png"></td>
</tr>
<tr>
<td>References</td><td>References source(s) used.</td><td><img src="images/References.png"></td>
</tr>
<tr>
<td>Likelyhood_of_Exploit</td><td>Gives the chance that the weakness would be taken advantage of using. Typically rated from low to high. </td><td><img src="images/References.png"></td>
</tr>
<tr>
<td>Weakness_Ordinalities</td><td>Ask Carlos about this one.</td><td><img src="images/Weakness_Ordinalities.png"></td>
</tr>
<tr>
<td>Relationship_Notes</td><td>Gives notes about the relationship.</td><td><img src="images/Relationship_Notes.png"></td>
</tr>
</table>   

# Conclusion

Now that we know which fields will be useful, we need to compare them in different ways. Four methods of comparison have been determined so far. They are specificity, subject, time, and purpose. The concept behind specificity is that with the hierarcy of <b>Weaknesses</b> that we have found, we can entries of similar specificity in order to see how similar they are. Subject is the fastest way to know if subjects are related. Also in the context of email, the subject is seperated in the subject line making it an invaluable asset. Time analyzes the data knowing that the way we communicate about a topic changes over time. This is due to a number of reasons ranging from an increased knowledge about a given topic to a new topic changes the way that we look at the given topic. Comparing the same entry accross versions will achieve this effect. Finally, purpose looks to compare the same field accross multiple documents. By analyzing the documents this way, we hopefully should be able to create a corpus that will give us an accurate similarity score.